# Import Dependencies

In [1]:
%pip install -e ..

Obtaining file:///home/tq/CodeProjects/BAMBOO
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for bamboo (pyproject.toml) ... done
  Created wheel for bamboo: filename=bamboo-0.1.0-0.editable-py3-none-any.whl size=5067 sha256=72d49086ac8997f85caa76592957fc4b993fee5b0bd90d8074e0010cabab54a8
  Stored in directory: /tmp/pip-ephem-wheel-cache-xv6lkc2g/wheels/f4/41/01/6acec7d7450f2330122df8c1063dfa3498ea166b8da912646d
Successfully built bamboo
  Attempting uninstall: bamboo
    Found existing installation: bamboo 0.1.0
    Uninstalling bamboo-0.1.0:
      Successfully uninstalled bamboo-0.1.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
from typing import List

import pandas as pd
from pydantic import BaseModel, Field

import bamboo

# Creating Example Dataset

This dataset contains retundant data; the enricher will automatically detect the fields used in the prompt and only run inference once.

In [3]:
base_data = [
    {"department": "sales", "severity": "Customer reported error when generating end-of-quarter sales forecast report; export button unresponsive and fails to produce downloadable PDF."},
    {"department": "sales", "severity": "Sales representative unable to update client opportunity notes; CRM fields appear locked after last software update despite correct user permissions."},
    {"department": "support", "severity": "User cannot reset account password; reset link email arrives blank and customer is unable to regain access despite multiple attempts today."},
    {"department": "support", "severity": "Client reports recurring chat disconnection every five minutes; issue persists across browsers, suggesting possible server-side session timeout misconfiguration."},
    {"department": "engineering", "severity": "Build pipeline fails during dependency installation stage; error logs indicate version mismatch between required libraries and current system environment."},
    {"department": "engineering", "severity": "Production API returning intermittent 500 errors under load; service health dashboard shows CPU spikes aligning with customer reports of downtime."},
]

data = base_data * 2  

df = pd.DataFrame(data)

df.sort_values(by="severity", inplace=True)

df

,department,severity
4,engineering,Build pipeline fails during dependency install...
10,engineering,Build pipeline fails during dependency install...
3,support,Client reports recurring chat disconnection ev...
9,support,Client reports recurring chat disconnection ev...
0,sales,Customer reported error when generating end-of...
6,sales,Customer reported error when generating end-of...
5,engineering,Production API returning intermittent 500 erro...
11,engineering,Production API returning intermittent 500 erro...
1,sales,Sales representative unable to update client o...
7,sales,Sales representative unable to update client o...


# Define Response Format

In [4]:
class Summary(BaseModel):
    verdict: str = Field(description="Categorization of 'Emergency', 'High', 'Medium', 'Low'")
    reason: str = Field(description="Reasoning for the categorization")

# Define Prompts

In [5]:
system_prompt = """
You are an assistant returning JSON only. 
Given a ticket description, help triage it in terms of severity and urgency.
"""

prompt_template = """
Classify tickets for routing. Consider historical policy by department and severity.\n
Department: {department}
Severity: {severity}
Return a JSON object with fields 'verdict' and 'reason'.
"""

# Enrich Dataset

In [6]:
df_out = df.bamboo.enrich(
    response_model=Summary,
    prompt_template=prompt_template,
    system_prompt=system_prompt,
    temperature=0.0,
    progress=True,
)

df_out

LLM enrich (unique): 100%|██████████| 6/6 [00:00<00:00, 10690.66it/s]


,department,severity,verdict,reason
4,engineering,Build pipeline fails during dependency install...,High,The build pipeline failure during the dependen...
10,engineering,Build pipeline fails during dependency install...,High,The build pipeline failure during the dependen...
3,support,Client reports recurring chat disconnection ev...,High,The recurring chat disconnection issue affects...
9,support,Client reports recurring chat disconnection ev...,High,The recurring chat disconnection issue affects...
0,sales,Customer reported error when generating end-of...,High,The issue directly impacts the sales team's ab...
6,sales,Customer reported error when generating end-of...,High,The issue directly impacts the sales team's ab...
5,engineering,Production API returning intermittent 500 erro...,Emergency,The production API is experiencing intermitten...
11,engineering,Production API returning intermittent 500 erro...,Emergency,The production API is experiencing intermitten...
1,sales,Sales representative unable to update client o...,High,The sales representative's inability to update...
7,sales,Sales representative unable to update client o...,High,The sales representative's inability to update...
